# Исследование базы данных книжного приложения

**Цель проекта:**

В нашем распоряжении доступ к базе данных книжного приложения, присутствует подробная документация и список необходимых заказчику задач для выполнения. Нашей конечной целью является проведения исследования для данных книжного приложения: знакомство с отношениями и атрибутами, наполнением таблиц, объёмом , а также поиском ответов необходимых заказчику (список необходимых задач приведён ниже). Итог каждого запроса необходимо сохранить в переменные для предоставления заказчику для дальнейших манипуляций.

**Задачи:**
- [Исследуйте таблицы — выведите первые строки, посчитайте количество строк в каждой таблице](#1);
- [Посчитайте, сколько книг вышло после 1 января 2000 года](#2);
- [Для каждой книги посчитайте количество обзоров и среднюю оценку](#3);
- [Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры](#4);
- [Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками](#5);
- [Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок](#6).

**Документация к базе данных:**

Таблица books:
Содержит данные о книгах.
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.


Таблица authors:
Содержит данные об авторах.
- author_id — идентификатор автора;
- author — имя автора.


Таблица publishers:
Содержит данные об издательствах.
- publisher_id — идентификатор издательства;
- publisher — название издательства.


Таблица ratings:
Содержит данные о пользовательских оценках книг.
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.


Таблица reviews:
Содержит данные о пользовательских обзорах на книги.
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

![описание базы данных](описание_бд.png)

Подключим необходимые для работы библиотеки:

In [1]:
# импортируем библиотеки
import pandas as pd   
import sqlalchemy as sa  # Импортируем SQLAlchemy для работы с БД через ORM (Object-Relational Mapping)

Создадим словарь, который будет хранить параметры для подключения к базе данных:

In [2]:
# устанавливаем параметры
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}

Сформируем строку подключения к БД в формате, который поддерживает PostgreSQL. Параметры берутся из словаря db_config и подставляются в соответствующие места.

In [3]:
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

Создадим объект engine с использованием метода create_engine из SQLAlchemy. 

Этот объект отвечает за управление соединением с базой данных. В параметре connect_args указано, что необходимо использовать SSL для безопасного подключения (sslmode='require').

In [4]:
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

Создадим функцию get_sql_data, которая выполняет SQL-запрос к базе данных.
Параметры функции: query (строка с SQL-запросом) и engine (объект подключения, по умолчанию используется ранее созданный).
Открывается соединение с базой данных с помощью контекстного менеджера with.
Выполняется SQL-запрос с помощью метода Pandas read_sql, результат возвращается в формате датафрейма.
По завершении выполнения запроса соединение автоматически закрывается.

In [5]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query: str, engine: sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:  # Открываем соединение
        return pd.read_sql(sql=sa.text(query), con=con)  # Выполняем запрос и возвращаем результат как DataFrame

Введём простой запрос для проверки корректности работы:

In [6]:
# формируем запрос и выводим данные
query = '''SELECT * FROM books LIMIT 3'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


Всё работает, начнём исследование!

## Исследуйте таблицы — выведите первые строки, посчитайте количество строк в каждой таблице.
<a id="1"></a>

Напишем для этого задания функцию которая выведет количество записей в таблицах и первые 5-ть записей в каждой таблице. Создание функции необходимо для того чтобы в дальнейшем если база данных пополнится иными таблицами, мы могли с лёгкостью перепровести подобную работу без сложных изменений в коде, а также в целом данная функция может пригодиться в других работах.

In [7]:
def display_table_info(sql_tables_name):
    """
    Функция для отображения количества записей и первых 5 строк в каждой таблице из списка sql_tables_name.
    
    Параметры:
    sql_tables_name (list): Список названий таблиц.
    """
    
    # Проходим по каждому имени таблицы
    for table_name in sql_tables_name:
        # Запрос вывода количества записей в таблице
        print(f'Результат выполнения запроса: найти количество записей в таблице {table_name}:')
        query = f'''SELECT COUNT(*) FROM {table_name}'''
        display(get_sql_data(query))  # Функция для выполнения SQL-запроса и отображения результата

        # Запрос вывода первых 5 записей
        print(f'\t\t\tПервые 5 записей таблицы - {table_name}:')
        query = f'''SELECT * FROM {table_name} LIMIT 5'''
        display(get_sql_data(query))
        
        # Пропуск двух строк для разделения вывода
        print()
        print()

In [8]:
# Вызов функции для 
sql_tables_name = ['books', 'authors', 'publishers', 'ratings', 'reviews'] # Названия имеющихся таблиц
display_table_info(sql_tables_name)

Результат выполнения запроса: найти количество записей в таблице books:


,count
0,1000


			Первые 5 записей таблицы - books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268




Результат выполнения запроса: найти количество записей в таблице authors:


...

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2




Результат выполнения запроса: найти количество записей в таблице reviews:


,count
0,2793


			Первые 5 записей таблицы - reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Первичное ознакомление с таблицами показало следующее:

1) Таблица books:
- количество записей = 1000;
- всего 6 атрибутов согласно документации и запросу;
- из особенностей: ничего, все записи были предсказуемыми на основе документации.

2) Таблица authors:
- количество записей = 636;
- всего 2 атрибута согласно документации и запросу;
- из особенностей: у наименования авторов свободный формат подачи - могут быть инициалы и фамилия, может быть имя и фамилия, могут быть фамилии через / и т.д.

3) Таблица publishers:
- количество записей = 340;
- всего 2 атрибута согласно документации и запросу;
- из особенностей: ничего, все записи были предсказуемыми на основе документации.

4) Таблица ratings:
- количество записей = 6456;
- всего 4 атрибута согласно документации и запросу;
- из особенностей: ничего, все записи были предсказуемыми на основе документации.

5) Таблица reviews:
- количество записей = 2793;
- всего 4 атрибута согласно документации и запросу;
- из особенностей: ничего, все записи были предсказуемыми на основе документации.

## Посчитайте, сколько книг вышло после 1 января 2000 года.
<a id="2"></a>

In [9]:
query = '''SELECT COUNT(*) AS total_books_after_2000
           FROM books
           WHERE publication_date > '2000-01-01';
'''
task_1 = get_sql_data(query)
display(task_1)

,total_books_after_2000
0,819


Задача №1

Формулировка задачи: Посчитайте, сколько книг вышло после 1 января 2000 года.

Ответ: после 1 января 2000 года вышло 819 книг.

Результат запроса лежит в переменной - task_1.

## Для каждой книги посчитайте количество обзоров и среднюю оценку.
<a id="3"></a>

Среднюю оценку мы округлим до 2-х знаков после запятой, а сортировку сделаем по убыванию количества обзоров.

In [10]:
query = '''SELECT b.title, 
                  COUNT(DISTINCT re.review_id) AS review_count, 
                  ROUND(AVG(ra.rating), 2) AS average_rating 
           FROM books b
           LEFT JOIN reviews re ON b.book_id = re.book_id
           LEFT JOIN ratings ra ON b.book_id = ra.book_id
           GROUP BY b.book_id
           ORDER BY review_count DESC;
'''
task_2 = get_sql_data(query)
display(task_2)

,title,review_count,average_rating
0,Twilight (Twilight #1),7,3.66
1,Water for Elephants,6,3.98
2,The Glass Castle,6,4.21
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,The Natural Way to Draw,0,3.00
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,Essential Tales and Poems,0,4.00


Задача №2

Формулировка задачи: Для каждой книги посчитайте количество обзоров и среднюю оценку.

Ответ: Среднюю оценку мы округлили до 2-х знаков после запятой, а сортировку сделали по убыванию количества обзоров, с результатом запроса можно ознакомиться с помощью переменной.

Результат запроса лежит в переменной - task_2.

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.
<a id="4"></a>

In [11]:
query = '''SELECT p.publisher, 
                  COUNT(b.book_id) AS book_count 
           FROM publishers p
           JOIN books b ON p.publisher_id = b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher_id
           ORDER BY book_count DESC
           LIMIT 1;

'''
task_3 = get_sql_data(query)
display(task_3)

,publisher,book_count
0,Penguin Books,42


Задача №3

Формулировка задачи: Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

Ответ: Издательство Penguin Books - выпустило наибольшее число книг толще 50 страниц.

Результат запроса лежит в переменной - task_3.

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.
<a id="5"></a>

In [12]:
query = '''WITH 
    book_with_50_plus_ratings AS (
        -- Находим книги с количеством оценок более 50
        SELECT book_id
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) > 50
    )
SELECT a.author_id, 
       a.author, 
       ROUND(AVG(r.rating), 3) AS avg_rating
FROM books b
-- Соединяем с книгами, у которых оценка 50+ 
INNER JOIN book_with_50_plus_ratings bw 
  ON b.book_id = bw.book_id
-- Соединяем с таблицей авторов
INNER JOIN authors a 
  ON b.author_id = a.author_id
-- Соединяем с оценками
INNER JOIN ratings r 
  ON b.book_id = r.book_id
GROUP BY a.author_id, a.author
ORDER BY AVG(r.rating) DESC
LIMIT 1;
'''
task_4 = get_sql_data(query)
display(task_4)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287


Задача №4

Формулировка задачи: Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

Ответ: Автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré, средняя оценка = 4.287

Результат запроса лежит в переменной - task_4.

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
<a id="6"></a>

In [13]:
query = '''WITH users_with_48_plus_ratings AS (
    -- Находим пользователей, оставивших более 48 оценок
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 48
)
SELECT ROUND(COUNT(DISTINCT rev.text) / COUNT(DISTINCT rev.username), 2) AS avg_review_count
FROM reviews AS rev
INNER JOIN users_with_48_plus_ratings AS u48
  ON rev.username = u48.username;

'''
task_5 = get_sql_data(query)
display(task_5)

,avg_review_count
0,24.0


Задача №5

Формулировка задачи: Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

Ответ: Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.

Результат запроса лежит в переменной - task_5.

**Общий вывод/ответы на все вопросы заказчика:**

- Задача №1

Формулировка задачи: Посчитайте, сколько книг вышло после 1 января 2000 года.

Ответ: после 1 января 2000 года вышло 819 книг.

Результат запроса лежит в переменной - task_1.



- Задача №2

Формулировка задачи: Для каждой книги посчитайте количество обзоров и среднюю оценку.

Ответ: Среднюю оценку мы округлили до 2-х знаков после запятой, а сортировку сделали по убыванию количества обзоров, с результатом запроса можно ознакомиться с помощью переменной.

Результат запроса лежит в переменной - task_2.


- Задача №3

Формулировка задачи: Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

Ответ: Издательство Penguin Books - выпустило наибольшее число книг толще 50 страниц.

Результат запроса лежит в переменной - task_3.


- Задача №4

Формулировка задачи: Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

Ответ: Автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré, средняя оценка = 4.287

Результат запроса лежит в переменной - task_4.



- Задача №5

Формулировка задачи: Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

Ответ: Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.

Результат запроса лежит в переменной - task_5.